In [12]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from collections import Counter

In [13]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df["label"] = iris.target
df["label"] = df["label"].map({0: "setosa", 1: "versicolor", 2: "virginica"})


In [14]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [15]:
def entropy(data):
    labels = data["label"]
    counts = Counter(labels)
    total = len(labels)
    return -sum((count/total) * np.log2(count/total) for count in counts.values())

In [16]:
def best_split(data, attributes):
    base_entropy = entropy(data)
    best_attr, best_threshold, best_gain_ratio = None, None, -1

    for attr in attributes:
        values = sorted(data[attr].unique())
        for i in range(len(values) - 1):
            threshold = (values[i] + values[i+1]) / 2
            left_split = data[data[attr] <= threshold]
            right_split = data[data[attr] > threshold]

            if len(left_split) == 0 or len(right_split) == 0:
                continue

            weighted_entropy = (len(left_split)/len(data))*entropy(left_split) + \ (len(right_split)/len(data))*entropy(right_split)
            info_gain = base_entropy - weighted_entropy

            split_info = 0
            for subset in [left_split, right_split]:
                if len(subset) > 0:
                    ratio = len(subset)/len(data)
                    split_info -= ratio * np.log2(ratio)

            if split_info > 0:
                gain_ratio = info_gain / split_info
            else:
                gain_ratio = 0

            if gain_ratio > best_gain_ratio:
                best_gain_ratio = gain_ratio
                best_attr = attr
                best_threshold = threshold

    return best_attr, best_threshold, best_gain_ratio

In [17]:
def c45(data, attributes, depth=0, max_depth=3):
    labels = data["label"]

    if len(set(labels)) == 1:
        return labels.iloc[0]
    if len(attributes) == 0 or depth == max_depth:
        return Counter(labels).most_common(1)[0][0]

    best_attr, best_threshold, best_gain_ratio = best_split(data, attributes)
    if best_attr is None:
        return Counter(labels).most_common(1)[0][0]

    left_split = data[data[best_attr] <= best_threshold]
    right_split = data[data[best_attr] > best_threshold]

    tree = {f"{best_attr} <= {best_threshold:.2f}": c45(left_split, attributes, depth+1, max_depth),
            f"{best_attr} > {best_threshold:.2f}": c45(right_split, attributes, depth+1, max_depth)}

    return tree

In [18]:
def predict(tree, sample):
    if not isinstance(tree, dict):
        return tree

    node = list(tree.keys())[0]

    if "<=" in node:
        attr, threshold = node.split(" <= ")
        threshold = float(threshold)
        if sample[attr] <= threshold:
            return predict(tree[node], sample)
        else:
            other_key = [k for k in tree.keys() if ">" in k][0]
            return predict(tree[other_key], sample)

    elif ">" in node:
        attr, threshold = node.split(" > ")
        threshold = float(threshold)
        if sample[attr] > threshold:
            return predict(tree[node], sample)
        else:
            other_key = [k for k in tree.keys() if "<=" in k][0]
            return predict(tree[other_key], sample)

In [19]:
attributes = list(df.columns[:-1])
tree = c45(train_df, attributes, max_depth=3)

print("Decision Tree (C4.5):")
print(tree)

Decision Tree (C4.5):
{'petal length (cm) <= 2.45': 'setosa', 'petal length (cm) > 2.45': {'petal width (cm) <= 1.75': {'petal length (cm) <= 5.35': 'versicolor', 'petal length (cm) > 5.35': 'virginica'}, 'petal width (cm) > 1.75': {'petal length (cm) <= 4.85': 'virginica', 'petal length (cm) > 4.85': 'virginica'}}}


In [20]:
correct = 0
for _, row in test_df.iterrows():
    pred = predict(tree, row)
    if pred == row["label"]:
        correct += 1

accuracy = correct / len(test_df)
print("\nTest Accuracy:", accuracy)


Test Accuracy: 1.0


In [21]:
sample = test_df.iloc[0]
print("\nTest Sample:", sample[:-1].to_dict())
print("True Label:", sample["label"])
print("Predicted:", predict(tree, sample))


Test Sample: {'sepal length (cm)': 6.1, 'sepal width (cm)': 2.8, 'petal length (cm)': 4.7, 'petal width (cm)': 1.2}
True Label: versicolor
Predicted: versicolor
